# Creation of database and loading of tables

In [2]:
require 'csv'
require 'sequel'

true

## Open connection to postgres

In [3]:
DB = Sequel.connect(adapter: 'postgres', 
                    host: 'postgres', 
                    user: 'postgres',
                    password: '*******'
                    )

#<Sequel::Postgres::Database: {:adapter=>"postgres", :host=>"postgres", :user=>"postgres", :password=>"******"}>

## Create a new database if not exists

In [4]:
result = DB.execute("SELECT datname FROM pg_catalog.pg_database WHERE datname = 'db_name'")
result == 0 ? DB.execute("CREATE DATABASE db_name") : "Database already exists"

"Database already exists"

In [5]:
Object.send(:remove_const, :DB) if defined?(DB)
DB = Sequel.connect(adapter: 'postgres', 
                    host: 'postgres', 
                    user: 'postgres',
                    password: '*******',
                    database: 'db_name'
                    )

#<Sequel::Postgres::Database: {:adapter=>"postgres", :host=>"postgres", :user=>"postgres", :password=>"******", :database=>"db_name"}>

## Create a new schema if not exists

In [6]:
result = DB.execute("SELECT schema_name FROM information_schema.schemata WHERE schema_name = 'schema_name'")
result == 0 ? DB.execute("CREATE SCHEMA schema_name") : "Schema already exists"

"Schema already exists"

In [7]:
Object.send(:remove_const, :DB) if defined?(DB)
DB = Sequel.connect(adapter: 'postgres', 
                    host: 'postgres', 
                    user: 'postgres',
                    password: '*******',
                    database: 'db_name',
                    search_path: ['schema_name']
                    )

#<Sequel::Postgres::Database: {:adapter=>"postgres", :host=>"postgres", :user=>"postgres", :password=>"*******", :database=>"db_name", :search_path=>["schema_name"]}>

## Create tables

In [14]:
create_tables_file = "postgres_create_tables.sql"
query = File.open(create_tables_file, "r") { |file| file.read }

# WARNING: if tables already exists, they will be dropped
DB.execute(query)

0

## Load models

In [12]:
load 'source_model.rb'

true

## Load tables data

In [15]:
# Converter to set headers to lowercase
converter = lambda { |header| header.downcase }

# Calculate progress during data loading
total_size = Dir['CSV/*'].select { |f| File.file?(f) }.sum { |f| File.size(f) }
progressive_size = 0

print("Loading data... 0%\n\n")

# Get files list and perform some actions for each file
files_list = Dir["CSV/*"]
files_list.each do |f|
  
  print("Loading " + f.split("/")[1] + "\n")
  data_file = CSV.read(f, headers: true, header_converters: converter)
  data_file.each do |row|
    # Insert rows in its table
    get_model(f.split("/")[1]).insert(row.to_hash)
  end
  
  # Update progress
  progressive_size = progressive_size + File.size(f)
  print("Loading data... " + ((progressive_size.to_f / total_size) * 100).to_i.to_s + "%\n\n")
  
end

print("Loading complete")

Loading data... 0%

Loading farmaci_biologici.csv
Loading data... 100%

Loading complete